# Лекция 11: параллелизм и Python

The free lunch is over? Not quite, we just need to be parallel now.

## ОС, процессы и потоки

### Общее описание

* Операционная система (ОС) - оболочка, организующая работу программ их взаимодействие с ресурсами компьютера.
* Процесс (process) - запущенная на выполнение программа, набор инструкций и текущее её состояние:
  * Образ машинного кода программы
  * Память (стек, куча)
  * Cостояние процессора (регистры, режимы работы и т.п.)
  * Текущие исполняемые команды
  * Дескрипторы ОС и файловые, права доступа
  * и т.д.
* Поток (thread) - поток выполнения
  * Обычно существует внутри процесса (процесс может иметь один или несколько потоков выполнения).
  * Описывается машинным кодом, контекстом родительского процесса (в т.ч. общие данные для отдельных потоков) и своим локальным контекстом (например, различное состояние процессора и стека для нескольких потоков одного процесса, собственные неразделяемые данные отдельного процесса (thread-local data)).
  * Имеет меньший чем у процесса описывающий контекст (более "легковесный").
  * За счёт этой "легковесности" и разделения контекста переключение между потоками одного процесса происходит быстрее, чем между отдельными процессами.
  * ОС считает наименьшей единицей исполнения задачи поток (если потоки поддерживаются, иначе процесс) и переключает выполнение между ими.
  * Кроме потоков ядра системы (kernel threads, управляемых ОС) можно иногда также рассматриваются пользовательские потоки (user-space threads), которые реализованы не в ОС, а с помощью библиотеки или конкретного интерпретатора/компилятора.
  * Thread-safe код - код умеющий работать в многопоточном окружении (или не требует синхронизаций, или есть все необходимые).

Процессы и потоки довольно похожи, некоторые ОС не разделяют эти понятия, а в тех, в которых они разделяются, поток обычно существует внутри процесса и несколько потоков внутри одного процесса могут обращаться к общим ресурсам.

Некоторые преимущества использования потоков:
* Программы могут оставаться отзывчивыми к пользовательским действиям, если интерфейс работает в отдельном потоке.
* Общие данные, с которыми можно работать из разных потоков.

### Возникающие проблемы и задачи

* Запуск новых процессов и потоков
* Диспетчеризация выполняющихся задач (scheduling)
* Доступ к общим ресурсам и обмен информацией между выполняющимися задачами (синхронизация состояния и передача данных)
* Обработка ошибок
* Завершение выполнения

В общем случае:
* Запуск процессов и потоков выполняется через API предоставляемое операционной системой либо через библиотеки, которые являются обертками вокруг этого API.
* Диспетчеризацию выполняет ОС, программа может косвенно на это влиять выставлением параметров и типа диспетчеризации через API либо использовать пользовательскую библиотеку, которая будет реализовывать диспетчеризацию поверх механизмов ОС.
* Программе доступны (напрямую через ОС или через оборачивающие библиотеки) механизмы синхронизации доступа к общим ресурсам, а также передачи сообщений между отдельными процессами и потоками.
* ОС предоставляет механизмы передачи и получения сообщений (сигналов и т.п.) об ошибках, а также их обработку. Также необходимо предусматривать механизмы для восстановления программы после ошибки в отдельных потоках или совместному корректному завершению программы.
* ОС предоставляет механизмы синхронизации и ожидания завершения выполнения, а также получения статуса после завершнения. Необходимо учитывать различные возникающие ситуации (одна из задач зависла, где-то была ошибка, надо дождаться полного завершения или нет и т.д.).

### Дополнительно про многопоточность и многопроцессность

* Хорошая книга про многопоточное программирование:
  * Herlihy, Shavit. The Art of Multiprocessor Programming. Elsevier, 2008

* Материалы первой лекции курса по параллельному программированию ШАД
  * Первые две видеозаписи https://yandexdataschool.ru/edu-process/courses/parallel

* Многопоточность в C++ 11 (в т.ч. для сравнения с тем, как это выглядит в Python)
  * Anthony Williams. C++ Concurrency in Action: Practical Multithreading. Manning, 2012

* Про различные механизмы взаимодействия между процессами в Unix системах (linux и т.д.)
  * Уильям Стивенс, Unix: взаимодействие процессов. Питер, 2003

## Многопоточность в Python

Python поддерживает многопоточность, API для этого реализовано в библиотеке threading. Однако многопоточность в Python своеобразна.

### Модуль threading

Для начала рассмотрим этот модуль стандартной библиотеки.

Модуль threading состоит из нескольких функций и переменных, класов представляющих собой поток, примитивы синхронизации, таймер периодического запуска и некоторых других элементов.

In [1]:
import threading

#### Класс Thread

Потоки представлены классом Thread:
* Он описывает API по запуску и завершению выполнения потоков.
* Он может служить обёрткой для пользовательской функции, которую нужно запустить на выполнение в потоке.
* От него можно наследоваться и перекрывать метод run(), чтобы описать свой поток.

#### Thread как обертка

In [2]:
def worker():
    """thread worker function"""
    print('Worker')
    return

threads = []
for i in range(5):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

WorkerWorker

Worker
Worker
Worker


Можно передать аргументы:

In [3]:
import threading

def worker(num):
    """thread worker function"""
    print('Worker: %s' % num)
    return

threads = []
for i in range(5):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

Worker: 0Worker: 1
Worker: 2
Worker: 3
Worker: 4



#### Определяем текущий поток

Нет нужды передавать аргументы, можно использовать функцию модуля threading:

In [4]:
import threading
import time

def worker():
    print(threading.current_thread().getName(), 'Starting')
    time.sleep(2)
    print(threading.currentThread().getName(), 'Exiting')

def my_service():
    print(threading.currentThread().getName(), 'Starting')
    time.sleep(3)
    print(threading.currentThread().getName(), 'Exiting')

t = threading.Thread(name='my_service', target=my_service)
w = threading.Thread(name='worker', target=worker)
w2 = threading.Thread(target=worker) # use default name

w.start()
w2.start()
t.start()

workerThread-14  Starting
my_service Starting
Starting


#### Логгирование и потоки

* С помощью модуля logging можно логировать информацию из потоков (не требует отдельной синхронизации т.к. thread-safe).
* Модуль умеет дописывать имя потока, если задать это в формате.

In [5]:
import logging
import threading
import time


# make it work in ipython
log_handler = logging.StreamHandler()
log_handler.setFormatter(
    logging.Formatter('[%(levelname)s] %(asctime)s (%(threadName)-10s) %(message)s')
)
logger = logging.getLogger()
logger.handlers = []
logger.addHandler(log_handler)
logger.setLevel(logging.DEBUG)

def worker():
    logging.debug('Starting')
    time.sleep(2)
    logging.debug('Exiting')

def my_service():
    logging.debug('Starting')
    time.sleep(3)
    logging.debug('Exiting')

t = threading.Thread(name='my_service', target=my_service)
w = threading.Thread(name='worker', target=worker)
w2 = threading.Thread(target=worker) # use default name

w.start()
w2.start()
t.start()
logging.debug('After')

[DEBUG] 2018-05-18 22:13:24,688 (worker    ) Starting
[DEBUG] 2018-05-18 22:13:24,690 (Thread-15 ) Starting
[DEBUG] 2018-05-18 22:13:24,690 (my_service) Starting
[DEBUG] 2018-05-18 22:13:24,690 (MainThread) After


#### Потоки демоны и не демоны

* По-умолчанию программы ждут, когда завершатся все созданные потоки.
* Иногда нужно, чтобы главный поток не ожидал завершения побочных.
* В таком случае можно дать потоку метку демона вызвав setDaemon(True).
* По-умолчанию этот флаг выключен.

In [6]:
def daemon():
    logging.debug('Starting')
    time.sleep(2)
    logging.debug('Exiting')

d = threading.Thread(name='daemon', target=daemon)
d.setDaemon(True)

def non_daemon():
    logging.debug('Starting')
    logging.debug('Exiting')

t = threading.Thread(name='non-daemon', target=non_daemon)

d.start()
t.start()
logging.debug('After')

[DEBUG] 2018-05-18 22:13:24,707 (daemon    ) Starting
[DEBUG] 2018-05-18 22:13:24,707 (non-daemon) Starting
[DEBUG] 2018-05-18 22:13:24,708 (MainThread) After
[DEBUG] 2018-05-18 22:13:24,709 (non-daemon) Exiting


Если запустить этот пример из файла, то последней строки не будет, т.к. главный поток не ждал завершения потока-демона.

Для ожидания завершения потока-демона можно воспользоваться функцией join():

In [7]:
def daemon():
    logging.debug('Starting')
    time.sleep(2)
    logging.debug('Exiting')

d = threading.Thread(name='daemon', target=daemon)
d.setDaemon(True)

def non_daemon():
    logging.debug('Starting')
    logging.debug('Exiting')

t = threading.Thread(name='non-daemon', target=non_daemon)

d.start()
t.start()

d.join()
t.join()

[DEBUG] 2018-05-18 22:13:24,738 (daemon    ) Starting
[DEBUG] 2018-05-18 22:13:24,741 (non-daemon) Starting
[DEBUG] 2018-05-18 22:13:24,742 (non-daemon) Exiting
[DEBUG] 2018-05-18 22:13:26,692 (worker    ) Exiting
[DEBUG] 2018-05-18 22:13:26,693 (Thread-15 ) Exiting
[DEBUG] 2018-05-18 22:13:26,710 (daemon    ) Exiting
[DEBUG] 2018-05-18 22:13:26,742 (daemon    ) Exiting


worker Exiting
Thread-14 Exiting


Тут может быть разница при запуске из файла прошлого и этого примера.

По-умолчанию join() блокируется на неограниченное время, но можно указать timeout:

In [8]:
def daemon():
    logging.debug('Starting')
    time.sleep(2)
    logging.debug('Exiting')

d = threading.Thread(name='daemon', target=daemon)
d.setDaemon(True)

def non_daemon():
    logging.debug('Starting')
    logging.debug('Exiting')

t = threading.Thread(name='non-daemon', target=non_daemon)

d.start()
t.start()

d.join(1)
logging.debug('d.isAlive() {}'.format(d.isAlive()))
t.join()

[DEBUG] 2018-05-18 22:13:26,772 (daemon    ) Starting
[DEBUG] 2018-05-18 22:13:26,772 (non-daemon) Starting
[DEBUG] 2018-05-18 22:13:26,777 (non-daemon) Exiting
[DEBUG] 2018-05-18 22:13:27,694 (my_service) Exiting
[DEBUG] 2018-05-18 22:13:27,773 (MainThread) d.isAlive() True


my_service Exiting


#### Обходим все потоки

In [9]:
import random

def worker():
    """thread worker function"""
    t = threading.currentThread()
    pause = random.randint(1,5)
    logging.debug('sleeping %s', pause)
    time.sleep(pause)
    logging.debug('ending')
    return

for i in range(3):
    t = threading.Thread(target=worker)
    t.setDaemon(True)
    t.start()

main_thread = threading.currentThread()
for t in threading.enumerate():
    if t is main_thread: # waiting for main thread causes deadlock
        continue
    logging.debug('joining %s', t.getName())
    t.join(1)

[DEBUG] 2018-05-18 22:13:27,818 (Thread-16 ) sleeping 5
[DEBUG] 2018-05-18 22:13:27,820 (Thread-17 ) sleeping 2
[DEBUG] 2018-05-18 22:13:27,825 (Thread-18 ) sleeping 5
[DEBUG] 2018-05-18 22:13:27,825 (MainThread) joining Thread-2
[DEBUG] 2018-05-18 22:13:28,776 (daemon    ) Exiting
[DEBUG] 2018-05-18 22:13:28,829 (MainThread) joining Thread-18
[DEBUG] 2018-05-18 22:13:29,827 (Thread-17 ) ending
[DEBUG] 2018-05-18 22:13:29,830 (MainThread) joining IPythonHistorySavingThread
[DEBUG] 2018-05-18 22:13:30,835 (MainThread) joining daemon
[DEBUG] 2018-05-18 22:13:30,836 (MainThread) joining Thread-16
[DEBUG] 2018-05-18 22:13:31,837 (MainThread) joining Thread-3
[DEBUG] 2018-05-18 22:13:32,825 (Thread-16 ) ending
[DEBUG] 2018-05-18 22:13:32,831 (Thread-18 ) ending
[DEBUG] 2018-05-18 22:13:32,840 (MainThread) joining Thread-17
[DEBUG] 2018-05-18 22:13:32,843 (MainThread) joining Thread-1


#### Наследуемся от Thread

In [10]:
class MyThread(threading.Thread):

    def run(self):
        logging.debug('running')
        return

for i in range(5):
    t = MyThread()
    t.start()

[DEBUG] 2018-05-18 22:13:33,858 (Thread-19 ) running
[DEBUG] 2018-05-18 22:13:33,858 (Thread-20 ) running
[DEBUG] 2018-05-18 22:13:33,862 (Thread-21 ) running
[DEBUG] 2018-05-18 22:13:33,863 (Thread-22 ) running
[DEBUG] 2018-05-18 22:13:33,863 (Thread-23 ) running


Можно перекрыть инициализацию и получить доступ к получаемым аргументам или начать поддерживать новые в своём классе-потомке Thread:

In [11]:
class MyThreadWithArgs(threading.Thread):

    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs=None):
        super().__init__(group=group, target=target, name=name)

        self.args = args
        self.kwargs = kwargs
        return

    def run(self):
        logging.debug('running with %s and %s', self.args, self.kwargs)
        return

for i in range(5):
    t = MyThreadWithArgs(args=(i,), kwargs={'a':'A', 'b':'B'})
    t.start()

[DEBUG] 2018-05-18 22:13:33,899 (Thread-24 ) running with (0,) and {'b': 'B', 'a': 'A'}
[DEBUG] 2018-05-18 22:13:33,900 (Thread-25 ) running with (1,) and {'b': 'B', 'a': 'A'}
[DEBUG] 2018-05-18 22:13:33,900 (Thread-26 ) running with (2,) and {'b': 'B', 'a': 'A'}
[DEBUG] 2018-05-18 22:13:33,901 (Thread-27 ) running with (3,) and {'b': 'B', 'a': 'A'}
[DEBUG] 2018-05-18 22:13:33,903 (Thread-28 ) running with (4,) and {'b': 'B', 'a': 'A'}


#### Потоки и таймеры

* Timer - поток, выполняющий функцию не сразу, а после некоторого ожидания.
* Такой поток можно отменить в любой момент до начала.

In [12]:
def delayed():
    logging.debug('worker running')
    return

t1 = threading.Timer(3, delayed)
t1.setName('t1')
t2 = threading.Timer(3, delayed)
t2.setName('t2')

logging.debug('starting timers')
t1.start()
t2.start()

logging.debug('waiting before canceling %s', t2.getName())
time.sleep(2)
logging.debug('canceling %s', t2.getName())
t2.cancel()
logging.debug('done')

[DEBUG] 2018-05-18 22:13:33,924 (MainThread) starting timers
[DEBUG] 2018-05-18 22:13:33,926 (MainThread) waiting before canceling t2
[DEBUG] 2018-05-18 22:13:35,932 (MainThread) canceling t2
[DEBUG] 2018-05-18 22:13:35,935 (MainThread) done


#### Синхронизация по Event

* Класс, позволяющий синхронизировать действия в различных потоках.
* set() - установить внутренний флаг.
* clear() - снять внутренний флаг.
* wait() - ждать, пока будет установлен флаг (set()).

In [13]:
def wait_for_event(e):
    """Wait for the event to be set before doing anything"""
    logging.debug('wait_for_event starting')
    event_is_set = e.wait()
    logging.debug('event set: %s', event_is_set)

def wait_for_event_timeout(e, t):
    """Wait t seconds and then timeout"""
    while not e.is_set():
        logging.debug('wait_for_event_timeout starting')
        event_is_set = e.wait(t)
        logging.debug('event set: %s', event_is_set)
        if event_is_set:
            logging.debug('processing event')
        else:
            logging.debug('doing other work')


e = threading.Event()
t1 = threading.Thread(name='block', 
                      target=wait_for_event,
                      args=(e,))
t1.start()

t2 = threading.Thread(name='non-block', 
                      target=wait_for_event_timeout, 
                      args=(e, 2))
t2.start()

logging.debug('Waiting before calling Event.set()')
time.sleep(3)
e.set()
logging.debug('Event is set')

[DEBUG] 2018-05-18 22:13:35,995 (block     ) wait_for_event starting
[DEBUG] 2018-05-18 22:13:35,995 (non-block ) wait_for_event_timeout starting
[DEBUG] 2018-05-18 22:13:35,995 (MainThread) Waiting before calling Event.set()
[DEBUG] 2018-05-18 22:13:36,926 (t1        ) worker running
[DEBUG] 2018-05-18 22:13:37,998 (non-block ) event set: False
[DEBUG] 2018-05-18 22:13:38,000 (non-block ) doing other work
[DEBUG] 2018-05-18 22:13:38,002 (non-block ) wait_for_event_timeout starting
[DEBUG] 2018-05-18 22:13:39,002 (MainThread) Event is set
[DEBUG] 2018-05-18 22:13:39,005 (block     ) event set: True
[DEBUG] 2018-05-18 22:13:39,006 (non-block ) event set: True


* Аргумент в wait(..) - на сколько секунд блокироваться.
* Возвращаемое значение - установлен ли проверяемый флаг.
* is_set() - неблокирущая проверка установленности флага.

#### Защита совместного доступа к ресурсам
* Стандартные типы данных защищены GIL.
* Но примитивные нет, а также пользовательские.
* При обращении из нескольких потоков могут возникать проблемы из-за одновременного доступа к данным.
* Общие данные нужно ограничивать.
* В модуле threading для этого можно воспользоваться классом Lock.

In [14]:
import threading
import random

class Counter(object):
    def __init__(self, start=0):
        self.lock = threading.Lock()
        self.value = start
    def increment(self):
        logging.debug('Waiting for lock')
        self.lock.acquire()
        try:
            logging.debug('Acquired lock')
            self.value = self.value + 1
        finally:
            self.lock.release()

def worker(c):
    for i in range(4):
        pause = random.random()
        logging.debug('Sleeping %0.02f', pause)
        time.sleep(pause)
        c.increment()
    logging.debug('Done')

counter = Counter()
for i in range(2):
    t = threading.Thread(target=worker, args=(counter,))
    t.start()

logging.debug('Waiting for worker threads')
main_thread = threading.currentThread()
for t in threading.enumerate():
    if t is not main_thread:
        # there are other ipython threads so limit join wait to avoid eternal waiting
        t.join(5) 
logging.debug('Counter: %d', counter.value)

[DEBUG] 2018-05-18 22:13:39,032 (non-block ) processing event
[DEBUG] 2018-05-18 22:13:39,088 (Thread-31 ) Sleeping 0.21
[DEBUG] 2018-05-18 22:13:39,089 (Thread-32 ) Sleeping 0.98
[DEBUG] 2018-05-18 22:13:39,091 (MainThread) Waiting for worker threads
[DEBUG] 2018-05-18 22:13:39,297 (Thread-31 ) Waiting for lock
[DEBUG] 2018-05-18 22:13:39,298 (Thread-31 ) Acquired lock
[DEBUG] 2018-05-18 22:13:39,299 (Thread-31 ) Sleeping 0.11
[DEBUG] 2018-05-18 22:13:39,409 (Thread-31 ) Waiting for lock
[DEBUG] 2018-05-18 22:13:39,410 (Thread-31 ) Acquired lock
[DEBUG] 2018-05-18 22:13:39,411 (Thread-31 ) Sleeping 0.20
[DEBUG] 2018-05-18 22:13:39,615 (Thread-31 ) Waiting for lock
[DEBUG] 2018-05-18 22:13:39,616 (Thread-31 ) Acquired lock
[DEBUG] 2018-05-18 22:13:39,618 (Thread-31 ) Sleeping 0.69
[DEBUG] 2018-05-18 22:13:40,076 (Thread-32 ) Waiting for lock
[DEBUG] 2018-05-18 22:13:40,079 (Thread-32 ) Acquired lock
[DEBUG] 2018-05-18 22:13:40,082 (Thread-32 ) Sleeping 0.46
[DEBUG] 2018-05-18 22:13:40,

С помощью acquire(False) можно просто узнать статус блокировки.

In [15]:
def lock_holder(lock, stop_event):
    logging.debug('Starting')
    while not stop_event.is_set():
        lock.acquire()
        try:
            logging.debug('Holding')
            time.sleep(0.5)
        finally:
            logging.debug('Not holding')
            lock.release()
        time.sleep(0.5)
    return
                    
def worker(lock):
    logging.debug('Starting')
    num_tries = 0
    num_acquires = 0
    while num_acquires < 3:
        time.sleep(0.5)
        logging.debug('Trying to acquire')
        have_it = lock.acquire(False)
        try:
            num_tries += 1
            if have_it:
                logging.debug('Iteration %d: Acquired',  num_tries)
                num_acquires += 1
            else:
                logging.debug('Iteration %d: Not acquired', num_tries)
        finally:
            if have_it:
                lock.release()
    logging.debug('Done after %d iterations', num_tries)


lock = threading.Lock()
stop_event = threading.Event()

holder = threading.Thread(target=lock_holder, args=(lock,stop_event), name='LockHolder')
holder.setDaemon(True)
holder.start()

worker = threading.Thread(target=worker, args=(lock,), name='Worker')
worker.start()

worker.join()
stop_event.set()

[DEBUG] 2018-05-18 22:13:59,214 (LockHolder) Starting
[DEBUG] 2018-05-18 22:13:59,215 (Worker    ) Starting
[DEBUG] 2018-05-18 22:13:59,215 (LockHolder) Holding
[DEBUG] 2018-05-18 22:13:59,717 (Worker    ) Trying to acquire
[DEBUG] 2018-05-18 22:13:59,718 (LockHolder) Not holding
[DEBUG] 2018-05-18 22:13:59,721 (Worker    ) Iteration 1: Not acquired
[DEBUG] 2018-05-18 22:14:00,226 (LockHolder) Holding
[DEBUG] 2018-05-18 22:14:00,233 (Worker    ) Trying to acquire
[DEBUG] 2018-05-18 22:14:00,237 (Worker    ) Iteration 2: Not acquired
[DEBUG] 2018-05-18 22:14:00,729 (LockHolder) Not holding
[DEBUG] 2018-05-18 22:14:00,741 (Worker    ) Trying to acquire
[DEBUG] 2018-05-18 22:14:00,744 (Worker    ) Iteration 3: Acquired
[DEBUG] 2018-05-18 22:14:01,232 (LockHolder) Holding
[DEBUG] 2018-05-18 22:14:01,246 (Worker    ) Trying to acquire
[DEBUG] 2018-05-18 22:14:01,249 (Worker    ) Iteration 4: Not acquired
[DEBUG] 2018-05-18 22:14:01,736 (LockHolder) Not holding
[DEBUG] 2018-05-18 22:14:01,75

#### Reentrant lock

* В случае с обычным Lock при попытке его захватить из того же потока будет провал
* Чтобы дать возможность в одном и том же потоке несколько раз захватывать лок, можно использовать класс RLock (reentrant lock).

In [16]:
lock = threading.Lock()

print('First try :', lock.acquire())
print('Second try:', lock.acquire(0))

First try : True
Second try: False


In [17]:
lock = threading.RLock()

print('First try :', lock.acquire())
print('Second try:', lock.acquire(0))
print(lock.acquire(0))

First try : True
Second try: True
True


#### Локи и контекстные менеджеры

In [18]:

def worker_with(lock):
    with lock:
        logging.debug('Lock acquired via with')
        
def worker_no_with(lock):
    lock.acquire()
    try:
        logging.debug('Lock acquired directly')
    finally:
        lock.release()

lock = threading.Lock()
w = threading.Thread(target=worker_with, args=(lock,))
nw = threading.Thread(target=worker_no_with, args=(lock,))

w.start()
nw.start()

[DEBUG] 2018-05-18 22:14:02,898 (Thread-33 ) Lock acquired via with


#### Синхронизация через condition

* Условная переменная.
* Для задач, когда нам нужно соединить потоки производителей и потребителей данных.
* Бывает полезно иметь возможность сигнализировать одному или нескольким потребителям.
* Класс Condition оборачивает Lock (или неявно создаёт, или ему можно передать).
* acquire() и release() вызывают соответствующие методы у обёрнутого лока.
* wait() отпускает лок и засыпает в ожидании notify() или notifyAll(), после чего пытается захватить лок.
* notify() - пробуждает один из ожидающих на wait() потоки.
* notifyAll() - пробуждает все ожидающие на wait() потоки.

In [19]:
def consumer(cond):
    """wait for the condition and use the resource"""
    logging.debug('Starting consumer thread')
    t = threading.currentThread()
    with cond:
        cond.wait()
        logging.debug('Resource is available to consumer')

def producer(cond):
    """set up the resource to be used by the consumer"""
    logging.debug('Starting producer thread')
    with cond:
        logging.debug('Making resource available')
        cond.notifyAll()

condition = threading.Condition()
c1 = threading.Thread(name='c1', target=consumer, args=(condition,))
c2 = threading.Thread(name='c2', target=consumer, args=(condition,))
p = threading.Thread(name='p', target=producer, args=(condition,))

c1.start()
time.sleep(2)
c2.start()
time.sleep(2)
p.start()

[DEBUG] 2018-05-18 22:14:02,922 (Thread-34 ) Lock acquired directly
[DEBUG] 2018-05-18 22:14:02,964 (c1        ) Starting consumer thread
[DEBUG] 2018-05-18 22:14:04,967 (c2        ) Starting consumer thread
[DEBUG] 2018-05-18 22:14:06,973 (p         ) Starting producer thread
[DEBUG] 2018-05-18 22:14:06,977 (p         ) Making resource available
[DEBUG] 2018-05-18 22:14:06,984 (c2        ) Resource is available to consumer
[DEBUG] 2018-05-18 22:14:06,989 (c1        ) Resource is available to consumer


#### Семафоры для допуска нескольких

* Семафор.
* Можно использовать для задач, когда нам нужно ограничить доступ к общему ресурсу, но можно пустить к нему несколько потоков.
* Пример: пул сетевых соединений или ограничение на количество одновременных загрузок.
* Имеет свой внутренний счётчик.
* Вызов acquire(), если счётчик больше 0, делает минус 1, иначе ждёт пока счётчик не станет больше нуля.
* Вызов release() увеличивает счётчик на единицу ("отдаёт обратно").

In [20]:
import threading

class ActivePool(object):
    def __init__(self):
        super(ActivePool, self).__init__()
        self.active = []
        self.lock = threading.Lock()
    def makeActive(self, name):
        with self.lock:
            self.active.append(name)
            logging.debug('Running: %s', self.active)
    def makeInactive(self, name):
        with self.lock:
            self.active.remove(name)
            logging.debug('Running: %s', self.active)

def worker(s, pool):
    logging.debug('Waiting to join the pool')
    with s:
        name = threading.currentThread().getName()
        pool.makeActive(name)
        time.sleep(0.1)
        pool.makeInactive(name)

pool = ActivePool()
s = threading.Semaphore(2)
for i in range(4):
    t = threading.Thread(target=worker, name=str(i), args=(s, pool))
    t.start()

[DEBUG] 2018-05-18 22:14:07,128 (0         ) Waiting to join the pool
[DEBUG] 2018-05-18 22:14:07,129 (1         ) Waiting to join the pool
[DEBUG] 2018-05-18 22:14:07,129 (0         ) Running: ['0']
[DEBUG] 2018-05-18 22:14:07,136 (1         ) Running: ['0', '1']
[DEBUG] 2018-05-18 22:14:07,131 (3         ) Waiting to join the pool
[DEBUG] 2018-05-18 22:14:07,129 (2         ) Waiting to join the pool


#### Приватные данные потока

* thread-specific data
* threading.local() создаёт объект, через который можно работать с такими данными.
* Не видны из соседнего потока.

In [21]:
import random

def show_value(data):
    try:
        val = data.value
    except AttributeError:
        logging.debug('No value yet')
    else:
        logging.debug('value=%s', val)


def worker(data):
    show_value(data)
    data.value = random.randint(1, 100)
    show_value(data)

local_data = threading.local()
show_value(local_data)
local_data.value = 1000
show_value(local_data)

for i in range(2):
    t = threading.Thread(target=worker, args=(local_data,))
    t.start()

[DEBUG] 2018-05-18 22:14:07,177 (MainThread) No value yet
[DEBUG] 2018-05-18 22:14:07,178 (MainThread) value=1000
[DEBUG] 2018-05-18 22:14:07,180 (Thread-35 ) No value yet
[DEBUG] 2018-05-18 22:14:07,182 (Thread-35 ) value=40
[DEBUG] 2018-05-18 22:14:07,183 (Thread-36 ) No value yet


Отнаследуемся, чтобы задать значение по-умолчанию:

In [22]:
def show_value(data):
    try:
        val = data.value
    except AttributeError:
        logging.debug('No value yet')
    else:
        logging.debug('value=%s', val)

def worker(data):
    show_value(data)
    data.value = random.randint(1, 100)
    show_value(data)

class MyLocal(threading.local):
    def __init__(self, value):
        logging.debug('Initializing %r', self)
        self.value = value

local_data = MyLocal(1000)
show_value(local_data)

for i in range(2):
    t = threading.Thread(target=worker, args=(local_data,))
    t.start()

[DEBUG] 2018-05-18 22:14:07,217 (Thread-36 ) value=77
[DEBUG] 2018-05-18 22:14:07,226 (MainThread) Initializing <__main__.MyLocal object at 0x7fb3f815fbe8>
[DEBUG] 2018-05-18 22:14:07,229 (MainThread) value=1000
[DEBUG] 2018-05-18 22:14:07,230 (Thread-37 ) Initializing <__main__.MyLocal object at 0x7fb3f815fbe8>
[DEBUG] 2018-05-18 22:14:07,230 (Thread-38 ) Initializing <__main__.MyLocal object at 0x7fb3f815fbe8>
[DEBUG] 2018-05-18 22:14:07,231 (Thread-37 ) value=1000
[DEBUG] 2018-05-18 22:14:07,232 (Thread-38 ) value=1000
[DEBUG] 2018-05-18 22:14:07,233 (Thread-37 ) value=1
[DEBUG] 2018-05-18 22:14:07,235 (Thread-38 ) value=7
[DEBUG] 2018-05-18 22:14:07,236 (0         ) Running: ['1']
[DEBUG] 2018-05-18 22:14:07,238 (1         ) Running: []
[DEBUG] 2018-05-18 22:14:07,238 (3         ) Running: ['3']
[DEBUG] 2018-05-18 22:14:07,239 (2         ) Running: ['3', '2']
[DEBUG] 2018-05-18 22:14:07,340 (3         ) Running: ['2']
[DEBUG] 2018-05-18 22:14:07,341 (2         ) Running: []


### Только один поток
* Стандартный интерпретатор берёт на себя синхронизацию доступа к общим данным потоков.
* Делает он это с помощью GIL (global interpreter lock) - общий для всего интерпретатора механизм, который позволяет единомоментно выполняться только одному потоку.
* Из-за этого факта выполнение в несколько потоков может быть медленнее выполнения в один поток (борьба за общий ресурс).
* Это верно не для всех интерпретаторов, поэтому если пишутся программы с использованием threading, то их надо писать как будто GIL нет (использовать примитивы синхронизации и т.п.), чтобы не было проблем выполнением на других интерпретаторах (или, возможно, в будущем).
* GIL не защищает от абсолютно всех случаев, некоторые действия могут не защищаться глобальным локом (работа с примитивными типами) и в этим моменты два потока могу работать параллельно (и могут быть ошибки, в т.ч. поэтому доступ к общим данным надо всё равно синхронизировать).

### Почему GIL

#### Особенности реализации

* Необходимо синхронизировать доступ к общим данным.
* Изначально при разработке интерпретатора CPython было решено использовать единый lock, который защищает все объекты.
* Это особенность реализации, есть интерпретаторы, которые не используют GIL.

#### Попытки убрать

В 1999 году для версии Python 1.5 пробовали реализовывать fine grained locking отдельных частей интерпретатора, но был неуспех.
  * однопоточное выполнение замедлилось на 40%
  * для двух потоков было ускорение
  * для более чем двух - не было заметного ускорения с ростом количества ядер

Главный вывод отсюда: убрать GIL - очень непростая задача. Многие части интерпретатора полагаются на глобальный lock и были написаны с учётом его.

#### Польза

Кроме этого, само многопоточное программирование - довольно сложная тема:
* Разум человека плохо приспособлем к осмыслению параллельных алгоритмов.
* Разработка и отладка многопоточных предложений на порядки сложнее, чем однопоточных.
* Таким образом, косвенно, GIL является своеобразной защитой программиста от себя самого защищая консистентность данных.
* Вместо многопоточности в Python принято использовать многопроцессность: в этой модели у задач нет общих данных (меньше проблем с доступом) и есть свои более изолированные механизмы синхронизации.

### Есть ли смысл в threading?

* С одной стороны, параллелизм через threading ограничен - одновременно будет выполняться только один поток.
* С другой, с помощью threading можно использовать concurrency в нашей программе.
* Вызовы некоторых функций скрывают за собой выполнение некоторых фоновых действий, во время которых программа просто ждёт.
* Примеры таких действий:
  * работа с файловой системой (особенно, если дисковая подсистема нагружена)
  * обмен данными по сети (особенно, если нагружена сеть или другие задачи на машине активно работают с сетью)
  * работа с механизмами ОС (системные вызовы, сигналы)
  * ожидание результата выполнения запущенной внешней задачи
* Пусть наша программа имеет части допускающие concurrent выполнение.
* Выделив отдельные логические потоки выполнения таких частей с помощью threading мы можем ускорить выполение программы.
* Пока один поток будет ждать результата некоторой операции, другие смогут выполнятся.

### Итого про многопоточность

* Python поддерживает работу с потоками через модуль threading стандартной библиотеки.
* В Python существует (верно для основного интерпретатора CPython и некоторых других) вынужденная синхронизация выполнения потоков через GIL (Global Interpreter Lock), одновременно на одном интерпретаторе выполняется только один поток.
* Использование многопоточности в Python имеет свои особенности и довольно ограничено по сравнению с другими языками, но имеет свою нишу: когда задачам нужно ждать результат выполнения внешних операций (чтение с диска, передача сообщений по сети, запросы в другие системы).

Ссылки на источники и почитать:

* Официальная документация по модулю threading https://docs.python.org/3.5/library/threading.html
* Источник про GIL https://jeffknupp.com/blog/2012/03/31/pythons-hardest-problem/
* Источник хороших примеров для модуля threading (там есть и для других) https://pymotw.com/3/threading/
* Ещё одна небольшая статья с примерами про потоки в Python http://www.python-course.eu/threads.php